In [1]:
import numpy as np
import scipy.stats as st

# B-S Formual  
We denote: v as option price, T as time to maturity, t as current time, s as stock price, r as risk free rate  
The price of European call option is given by
$$
V_{c}(t,S_{t}) = S_{t} N(d_{1}(t,S_{t})) - \exp{(-r(T-t))} K N(d_{2}(t,S_{t}))
$$
where 
$$
d_{1}(t,S_{t}) = \frac{1}{\sigma \sqrt{T-t}}(\log{\frac{S_{t}}{K}} + (r + \frac{1}{2} \sigma^2)(T-t))
$$
$$
d_{2}(t,S_{t}) = d_{1}(t,S_{t}) - \sigma\sqrt{T-t}
$$
N is the cumulative distribution function for standard normal distribution

Similarly, the price of European put is given by
$$
V_{p}(t,S_{t}) = \exp{(-r(T-t))} K N(-d_{2}(t,S_{t})) - S_{t} N(-d_{1}(t,S_{t}))
$$

vega of call 
$$
Vega = \frac{\partial V_{c}(t,S_{t})}{\partial \sigma} = S_{t} \phi(d_{1}) \sqrt{T-t}
$$

vega of put
$$
Vega = \frac{\partial V_{p}(t,S_{t})}{\partial \sigma} = \exp{(-r(T-t))} K \phi(d_{2}) \sqrt{T-t}
$$


where $\phi{}$ is the probability density function of normal distribution  
when I was deriving vega of call/put, I also surprisingly found:

$$
S_{t}\phi(d1) = k\exp(-rt)\phi(d2)
$$


In [2]:
def newton_solver(f,df,x,tol = 1e-6,max_loop = 1000,return_list = False):
    root_trials = []
    if abs(f(x)) <= tol:
        return x
    loop = 0
    while abs(f(x))> tol and loop < max_loop:
        if abs(df(x)) < tol:
            print('The slope of function @ {} is 0'.format(x))
            return None
        x = x - float(f(x)/df(x))
        root_trials.append(x)
        loop += 1
        print('{} trials, trial = {}'.format(loop,x))
    
    if loop >= max_loop:
        print('No solution!')
        return None
    print('Get root after {} iterations'.format(loop))
    if return_list:
        return root_trials[-1],root_trials
    else:          
        return root_trials[-1]

In [3]:
def dV_dsigma(S_0,K,sigma,tau,r):
    #parameters and value of Vega
    d2   = (np.log(S_0 / float(K)) + (r - 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    value = K * np.exp(-r * tau) * st.norm.pdf(d2) * np.sqrt(tau)
    return value

def BS_Call_Option_Price(CP,S_0,K,sigma,tau,r):
    #Black-Scholes Call option price
    d1    = (np.log(S_0 / float(K)) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)
    if str(CP).lower()=="c" or str(CP).lower()=="1":
        value = st.norm.cdf(d1) * S_0 - st.norm.cdf(d2) * K * np.exp(-r * tau)
    elif str(CP).lower()=="p" or str(CP).lower()=="-1":
        value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S_0
    return value

In [4]:
def impliedVol_calculator(CP,S_0,K,tau,r,market_price,sigmaini = 0.5,):
    optPrice = lambda sigma:BS_Call_Option_Price(CP,S_0,K,sigma,tau,r) ## option price wrt sigma
    vega = lambda sigma:dV_dsigma(S_0,K,sigma,tau,r) # vega wrt sigma

    fx = lambda sigma:optPrice(sigma) - market_price
    dfx_dx = vega

    result = newton_solver(fx,dfx_dx,sigmaini,tol = 1e-6,max_loop = 1000,return_list = False)
    return result

In [5]:
V_market = 2    # market call option price
K        = 120  # strike
tau      = 1    # time-to-maturity
r        = 0.05 # interest rate
S_0      = 100  # today's stock price
sigmaInit    = 0.25  # Initial implied volatility
CP       ="c" #C is call and P is put

In [7]:


sigma_imp = impliedVol_calculator(CP,S_0,K,sigmaInit,tau,r,V_market)
print('''Implied volatility for CallPrice= {}, strike K={}, 
      maturity T= {}, interest rate r= {} and initial stock S_0={} 
      equals to sigma_imp = {:.7f}'''.format(V_market,K,tau,r,S_0,sigma_imp))

# Check! 
val = BS_Call_Option_Price(CP,S_0,K,sigma_imp,tau,r)
print('Option Price for implied volatility of {0} is equal to {1}'.format(sigma_imp, val))

1 trials, trial = 0.17953469237708541
2 trials, trial = 0.1620128111187268
3 trials, trial = 0.16148330309570028
4 trials, trial = 0.16148272884207326
Get root after 4 iterations
Implied volatility for CallPrice= 2, strike K=120, 
      maturity T= 1, interest rate r= 0.05 and initial stock S_0=100 
      equals to sigma_imp = 0.1614827
Option Price for implied volatility of 0.16148272884207326 is equal to 2.0000000000206164


# How to calculate volatility smile and volatility surface?
- 1. when calculating volatility smile, we should use options with same maturity and different strikes
- 2. when calcluating volatility surface, we are expected to see the following pic

![image info]('pics/impliedVolatility.png')

<img src="C:/Users/Ruich/Desktop/Recent/NumericalMethodFinance/pics/impliedVolatility.png">